In [43]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle
from collections import Counter
from sklearn.neighbors import NearestNeighbors
import cv2 

In [44]:
feature_extraction = []
filename = ['embeddings0-20.pkl' , 'embeddings20-30.pkl' ,'embeddings30-44.441.pkl']

for file in filename :
    with open(file , 'rb') as f : 
        data = pickle.load(f)
        for i in range(len(data)):
            feature_extraction.append(data[i])

print(len(feature_extraction))

filenames = []
with open ('filenames.pkl' , 'rb') as f:
    filenames = pickle.load(f)

print(filenames[:10])



44441
['images/15925.jpg', 'images/18260.jpg', 'images/31891.jpg', 'images/26635.jpg', 'images/7942.jpg', 'images/46102.jpg', 'images/59657.jpg', 'images/19367.jpg', 'images/15189.jpg', 'images/57352.jpg']


In [45]:
print(np.array(feature_extraction).shape)

(44441, 2048)


In [46]:
model = ResNet50(weights='imagenet' , include_top=False , input_shape=(224,224,3))
model.trainable = False 
model = tensorflow.keras.Sequential([
    model ,
    GlobalMaxPooling2D()
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d_3 (Gl  (None, 2048)              0         
 obalMaxPooling2D)                                               
                                                                 
Total params: 23587712 (89.98 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [52]:
def extract_features(img_path , model):
    img = image.load_img(img_path , target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img , axis = 0)
    img = preprocess_input(img)
    result = model.predict(img).flatten()
    result = result / norm(result)
    return result  

In [54]:
features = extract_features('test1.jpg' , model)
features

1/1 [==============================] - 0s 101ms/step


array([0.00185183, 0.03264625, 0.00192974, ..., 0.00556849, 0.03557047,
       0.04254084], dtype=float32)

In [55]:
neighbours = NearestNeighbors(n_neighbors=6,algorithm='brute',metric = 'euclidean')
neighbours.fit(feature_extraction)
distances , indices = neighbours.kneighbors([features])
print(indices)

[[34478 15696  4868 21406 29476  4476]]


In [58]:
for file in indices[0][1:]:
    print(filenames[file])
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output' , cv2.resize(temp_img , (128,128)))
    cv2.waitKey(0)

images/2212.jpg
images/2207.jpg
images/2478.jpg
images/2242.jpg
images/2240.jpg
